<a href="https://colab.research.google.com/github/virf96/Chat-Bot/blob/master/DatosFaltantes_SustitucionMuestraAleatoria_FeatureEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sustitución por Muestra Aleatoria ==> Feature-engine

### Qué es Feature-engine?

Feature-engine es una librería de Python que hemos creado para este curso. 

- Feature-engine incluye todas las técnicas de ingeniería de variables descritas en este curso
- Feature-engine funciona como Scikit-learn, por lo tanto es fácil de aprender
- Feature-engine te permite implementar pasos de ingeniería de variables específicos para diferentes grupos de variables
- Feature-engine puede ser integrado con las pipelines de Scikit-learn pipeline permitiendo construir modelos fácilmente
** Feature-engine te permite diseñar y guardar un flujo de ingeniería de variables con procesos diseñados específicamente para diferentes grupos de variables.**

-------------------------------------------------------------------
Feature-engine puede ser instalado vía pip ==> pip install feature-engine

- Asegurate que haz instalado Feature-engine antes de correr este notebook

Para más detalle visita el [website the trainindata]( https://www.trainindata.com/feature-engine) 


## En este demo:

Vamos a usar **Feature-engine para hacer la sustitución por muestra aleatoria** usando los datos Ames House Price.

### Nota: 
* 'Imputer' se deriva del verbo en inglés 'to impute' que quiere decir sustituir o reemplazar. Imputer es el objeto que completa la sustitución, de ahí el nombre dado a la clase.


In [2]:
pip install feature_engine

     |████████████████████████████████| 9.5MB 5.8MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

#  feature engine
from feature_engine import missing_data_imputers as mdi

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# carguemos los datos con unas columnas seleccionadas

cols_to_use = [
    'BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt',
    'SalePrice'
]

data = pd.read_csv('/content/drive/My Drive/datasets/houseprice.csv', usecols=cols_to_use)
data.head()

,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,GarageYrBlt,SalePrice
0,65.0,196.0,Gd,NaN,2003.0,208500
1,80.0,0.0,Gd,TA,1976.0,181500
2,68.0,162.0,Gd,TA,2001.0,223500
3,60.0,0.0,TA,Gd,1998.0,140000
4,84.0,350.0,Gd,TA,2000.0,250000


In [7]:
#Verificamos valores nulos
data.isnull().mean()

LotFrontage    0.177397
MasVnrArea     0.005479
BsmtQual       0.025342
FireplaceQu    0.472603
GarageYrBlt    0.055479
SalePrice      0.000000
dtype: float64

In [8]:
# separar datos en segmentos entrenamiento y prueba
#Esto porque solo debemos aprender los datos de imputación del set de entrenamiento
# primero, separemos el target (SalePrice) del resto de las variables

cols_to_use.remove('SalePrice')

X_train, X_test, y_train, y_test = train_test_split(data[cols_to_use],
                                                    data['SalePrice'],
                                                    test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((1022, 5), (438, 5))

## Feature-engine Random Sampler por defecto captura todas las variables

In [9]:
# llamemos el imputer de Feature-engine
# no necesitamos especificar nada
#Solo indicamos la semilla aleatoria
imputer = mdi.RandomSampleImputer(random_state = 29)

In [11]:
# ajustemos el imputer al set de entrenamiento

imputer.fit(X_train)

RandomSampleImputer(random_state=29, seed='general', seeding_method='add',
                    variables=['BsmtQual', 'FireplaceQu', 'LotFrontage',
                               'MasVnrArea', 'GarageYrBlt'])

In [12]:
# vemos que el imputer encontró las variables categóricas 
# y numéricas

imputer.variables

['BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [13]:
# el imputer guarda una copia de las variables seleccionadas del
# segmento de entrenamiento, de las cuales extraer la muestra aleatoria

imputer.X_.head()

,BsmtQual,FireplaceQu,LotFrontage,MasVnrArea,GarageYrBlt
64,Gd,NaN,NaN,573.0,1998.0
682,Gd,Gd,NaN,0.0,1996.0
960,TA,NaN,50.0,0.0,NaN
1384,TA,NaN,60.0,0.0,1939.0
1100,TA,NaN,60.0,0.0,1930.0


In [14]:
# feature engine devuelve un dataframe

tmp = imputer.transform(X_train)
tmp.head()

,BsmtQual,FireplaceQu,LotFrontage,MasVnrArea,GarageYrBlt
64,Gd,TA,60.0,573.0,1998.0
682,Gd,Gd,90.0,0.0,1996.0
960,TA,Gd,50.0,0.0,1977.0
1384,TA,Gd,60.0,0.0,1939.0
1100,TA,Gd,60.0,0.0,1930.0


In [15]:
#revisemos que ya no tenemos valores nulos

tmp[imputer.variables].isnull().mean()

BsmtQual       0.0
FireplaceQu    0.0
LotFrontage    0.0
MasVnrArea     0.0
GarageYrBlt    0.0
dtype: float64

Revisa la documentación del RandomSampleImputer() para aprender cómo poner semillas dependiendo de variables en el set de datos, como explicamos anteriormente:
https://feature-engine.readthedocs.io/en/latest/imputers/RandomSampleImputer.html